# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Markers, please use your onw API keys to run the script, thank you!
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
csv_path = "../output_files/cities_weather.csv"
cities_weather_df = pd.read_csv(csv_path)

cities_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Upernavik,72.7868,-56.1549,35.78,41,0,22.39,GL,1631414830
1,Barrow,71.2906,-156.7887,44.62,76,20,4.61,US,1631414804
2,Mar,43.4002,-4.0094,65.01,98,100,0.00,ES,1631414831
3,Gulmarg,34.0490,74.3921,54.37,84,94,1.16,IN,1631414832
4,Guerrero,17.6667,-100.0000,67.69,95,100,3.22,MX,1631414833
...,...,...,...,...,...,...,...,...,...
537,Chauk,20.8833,94.8167,79.34,83,100,9.01,MM,1631415361
538,Menongue,-14.6585,17.6910,57.24,48,7,3.83,AO,1631415362
539,Montepuez,-13.1256,38.9997,65.12,81,0,7.23,MZ,1631415362
540,Erice,38.0370,12.5865,66.67,73,0,8.23,IT,1631415363


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure the gmaps
gmaps.configure(api_key = g_key)

# Create variable to store Latitude and Longitude
city_lat_lng = cities_weather_df[["Lat", "Lng"]]
humidity = cities_weather_df["Humidity"].astype(float)

# Create figure layout options in dictionary
figure_layout = {"width": "100%",
                 "height": "600px",
                 "padding": "1px"}

# Create heat layer
fig = gmaps.figure(layout = figure_layout, center = (20.0, 10.0), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(city_lat_lng,
                                 weights = humidity,
                                 max_intensity = np.max(humidity),
                                 dissipating = False,
                                 point_radius = 2)

# add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create dataframe for ideal locations with cities meets the requirements:
# A max temperature between 70(F) - 80(F)
# Wind speed less than 10 mph
# Zero cloudiness
vacation_cities_df = cities_weather_df.loc[(cities_weather_df["Max Temp"] >= 70) &
                                           (cities_weather_df["Max Temp"] <= 80) &
                                           (cities_weather_df["Wind Speed"] <= 10) &
                                           (cities_weather_df["Cloudiness"] == 0)].dropna()

vacation_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
110,Kamaishi,39.2667,141.8833,79.75,62,0,7.11,JP,1631414939
173,Mīnāb,27.1467,57.0801,78.30,36,0,3.29,IR,1631414998
224,Havelock,34.8791,-76.9013,72.95,93,0,2.48,US,1631415068
232,Camas,37.4020,-6.0331,71.96,86,0,1.01,ES,1631415076
289,Miyako,39.6367,141.9525,79.23,56,0,4.79,JP,1631415129
317,San Fernando del Valle de Catamarca,-28.4696,-65.7852,71.78,42,0,1.99,AR,1631415155
336,Pishin,30.5803,66.9961,74.41,28,0,1.39,PK,1631415171
406,Linxia Chengguanzhen,35.6003,103.2064,72.64,49,0,3.58,CN,1631415244
412,Corsicana,32.0954,-96.4689,76.77,58,0,1.01,US,1631415248
464,Abnūb,27.2696,31.1511,73.62,58,0,8.97,EG,1631415298


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Drop un-needed columns from the original data frame
hotel_df = vacation_cities_df.loc[:, ["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""

# Base url for Google Places API - Nearby Search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set parameters for the API call
params = {"keyword": "hotel",
          "type": "lodging",
          "key": g_key,
          "radius": 5000,
          "location": ""}

# Perform API calls to collect hotel name
for index, row in hotel_df.iterrows():
    
    # Insert city coordinates into parameters
    params["location"] = str(row[1]) +","+ str(row[2])
    
    # API call
    response = requests.get(base_url, params).json()
    
    # Store into results part of dictionary
    hotel_results = response["results"]
    
    # Print hotel name and add to data frame if hotel is successfully found
    try:
        print(f"{row['City']}, {row['Country'].upper()} hotel: {hotel_results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = hotel_results[0]["name"]
    
    # Print error statement if hotels cannot be found
    except (KeyError, IndexError):
        print(f"No hotel found for this city within 5000 meters.")
    
hotel_df

Kamaishi, JP hotel: Hotel Folkloro Sanriku Kamaishi
Mīnāb, IR hotel: Minab Tourism Hotel
Havelock, US hotel: Hampton Inn Havelock
Camas, ES hotel: Hotel Casa 1800 Seville
Miyako, JP hotel: The Park Hotel Jodogahama
San Fernando del Valle de Catamarca, AR hotel: Hotel Casino Catamarca
Pishin, PK hotel: Ali Khan Hotel
Linxia Chengguanzhen, CN hotel: Linxia Hotel
Corsicana, US hotel: Best Western Executive Inn
No hotel found for this city within 5000 meters.
No hotel found for this city within 5000 meters.


,City,Lat,Lng,Country,Hotel Name
110,Kamaishi,39.2667,141.8833,JP,Hotel Folkloro Sanriku Kamaishi
173,Mīnāb,27.1467,57.0801,IR,Minab Tourism Hotel
224,Havelock,34.8791,-76.9013,US,Hampton Inn Havelock
232,Camas,37.4020,-6.0331,ES,Hotel Casa 1800 Seville
289,Miyako,39.6367,141.9525,JP,The Park Hotel Jodogahama
317,San Fernando del Valle de Catamarca,-28.4696,-65.7852,AR,Hotel Casino Catamarca
336,Pishin,30.5803,66.9961,PK,Ali Khan Hotel
406,Linxia Chengguanzhen,35.6003,103.2064,CN,Linxia Hotel
412,Corsicana,32.0954,-96.4689,US,Best Western Executive Inn
464,Abnūb,27.2696,31.1511,EG,


In [6]:
cities_count = hotel_df["City"].nunique()
print(f"%s cities match ideal vaction city requirements." % cities_count)

11 cities match ideal vaction city requirements.


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))